In [2]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

args=Namespace(rho=0,
               seed=9384,
              npop=1e4,
              mu=1e-08,
              m=0.05,
              ndemes=36,
              tmove=100,
              ss=10)

In [3]:
import msprime
import numpy as np
import statistics
import math
import allel
import pandas as pd
import statsmodels.api as sm
from scipy import (stats,ndimage)

In [4]:
#number of demes
d=args.ndemes

# dimension of square grid
dim=int(np.sqrt(d))

# define 2d grid to with deme identity
pmat=np.arange(0,d).reshape(dim,dim)

In [5]:
#define function to generate adjacency matrix
#arguments:
#m = migration rate in one direction
#nd = number of demes
def step_mig_mat(m,nd):
    #m is the uni-directional symmetric migration rate
    #NOTE: nd MUST be a squared number
    if(math.sqrt(nd).is_integer()==False):
        raise ValueError("nd must be a squared number (e.g. 4, 9, 16 ...) for the 2D model")
    else:
        nd2=int(math.sqrt(nd))
        #create matrix which will be used to determine which cells are adjacent in 2-dimensions
        #diagonals not considered for now but can be incorporated later if needed
        pmat=np.arange(0,nd).reshape(nd2,nd2)

        #create empty migration matrix to be filled in. This will be the output
        mmat=np.zeros(shape=[ (nd+3),(nd+3) ])

        #go through each cell in pmat and find out which cells are adjacent
        #first define functions to take care of corners and sides
        def contain(ix,max_ix):
            if ix<0:
                return(0)
            if ix>(max_ix-1):
                return(max_ix-1)
            else:
                return(ix)

        for ii in range(nd):
            center_ix=np.where(pmat==ii)
            top_ix=pmat[contain(center_ix[0]-1,nd2),contain(center_ix[1],nd2)]
            bottom_ix=pmat[contain(center_ix[0]+1,nd2),contain(center_ix[1],nd2)]
            left_ix=pmat[contain(center_ix[0],nd2),contain(center_ix[1]-1,nd2)]
            right_ix=pmat[contain(center_ix[0],nd2),contain(center_ix[1]+1,nd2)]

            mmat[ii,top_ix]=mmat[ii,bottom_ix]=mmat[ii,left_ix]=mmat[ii,right_ix]=m
            mmat[top_ix,ii]=mmat[bottom_ix,ii]=mmat[left_ix,ii]=mmat[right_ix,ii]=m

            mmat[ii,ii]=0

    return(mmat)

#generate migration matrix with migration rate provided by user
mig_mat=step_mig_mat(m=0.05,nd=d)

In [9]:
#set up the initial population
ss_each=args.ss*2
sample_sizes=[ss_each]*36

population_configurations = [
msprime.PopulationConfiguration(sample_size=k)
for k in sample_sizes]

population_configurations.extend([msprime.PopulationConfiguration(sample_size=0)]*3)


In [10]:
############ set up the demography

#move all northern lineages
demog=[msprime.MassMigration(
    time=100, source=i, destination=0, proportion=1.0) for i in range(1,39)]

ts=msprime.simulate(Ne=args.npop,
                      population_configurations=population_configurations,
                      migration_matrix=mig_mat,
                      mutation_rate=1e-08,
                      recombination_rate=0,
                      length=1e6,
                   demographic_events=demog)

In [11]:
############ set up the demography

#move all northern lineages
demog=[msprime.MassMigration(
    time=100, source=i, destination=0, proportion=1.0) for i in range(1,30)]

#move all southern lineages
demog.extend([msprime.MassMigration(time=100, source=i, destination=35, proportion=1.0) for i in range(30,35)])

demog_adds=[
#change migration rate b/w all demes to 0
msprime.MigrationRateChange(time=100,rate=0),

#t 4500: Migrate lineages from 35 > 36 (WHG-south to Steppe) & from 0 > 36 (WHG1 to steppe)
msprime.MassMigration(
        time=4500, source=35, destination=36, proportion=0.2),
        
msprime.MassMigration(
        time=4501, source=0, destination=36, proportion=0.5),

#t 7510: Migrate lineages from 35 > 38 (WHG-south > EF) & 0>38 (WHG-n > EF)
msprime.MassMigration(
        time=7510, source=35, destination=38, proportion=0.75),
        
msprime.MassMigration(
        time=7511, source=0, destination=38, proportion=0.4),

#t 9000: Migrate lineages 36 > 37: steppe to EF & WHG1 and WHG2 merge
msprime.MassMigration(
        time=9000, source=36, destination=37, proportion=0.5),
        
msprime.MassMigration(
        time=9001, source=35, destination=0, proportion=1),

#t 25k: Migrate lineages 36 > 38, steppe & EF merge
msprime.MassMigration(
        time=25000, source=36, destination=38, proportion=1),

#t 30k: Migrate lineages 0 > 37: HG and basal Eurasians merge
msprime.MassMigration(
        time=30000, source=0, destination=37, proportion=1),

#t 45k: Migrate lineages 37 > 38: HG and basal Eurasians merge    
msprime.MassMigration(
        time=45000, source=37, destination=38, proportion=1)]

demog.extend(demog_adds)



In [12]:
%%time
ts=msprime.simulate(Ne=args.npop,
                      population_configurations=population_configurations,
                      migration_matrix=mig_mat,
                      mutation_rate=1e-08,
                      recombination_rate=0,
                      length=1e6,
                   demographic_events=demog)

CPU times: user 96.1 ms, sys: 2.89 ms, total: 99 ms
Wall time: 98.2 ms
